In [3]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pprint

In [4]:
r = requests.get('https://www.sanfoundry.com/1000-microbiology-questions-answers/')
c = r.content

In [5]:
soup = BeautifulSoup(c, "html5lib")
ec = soup.find("div", class_="entry-content")
ads = ec.find_all("div", id=re.compile("^sf-ads")) #gather all ads
vans = ec.find_all("span", class_="collapseomatic") #gather all collapse spans - view answer button
brs = ec.find_all("br") #gather all br tags
for ad in ads:
    ad.decompose() #removes all ads
for van in vans:
    van.decompose() #removes all collapse spans - view answer button
for br in brs:
    br.decompose() #removes all br tags
# print(ec.prettify())
ps = ec.find_all('p')[1:]
# print(ps)
questions = []
dict_ans_idx = {
            'a': 1,
            'b': 2,
            'c': 3,
            'd': 4
        } #map letter numbering of option to its index in content
for p in ps:
    try:
        contents = p.contents
#         print(contents)
#         print(len(contents))
        answer = p.find_next_sibling("div", class_="collapseomatic_content")
#         print(answer.contents)
        ans, exp = answer.contents[:2]
#         print(ans, exp)
        ans_let = ans.split(':')[1].strip().lower()
#         print(ans_let)
    #         print('?????', "==========")
#         print(dict_ans_idx[ans_let], ".......")
        question = contents[0].split('.')[1].lstrip()
#         print(question)
#         print(len(question))
        options = contents[1:]
        options.pop() #remove the last newline char in the list
#         print(options)
        correct_answer = options.pop(dict_ans_idx[ans_let]-1) # remove correct answer
#         print(correct_answer)
        correct_answer = correct_answer.split(')',1)[1].lstrip() 
#         print(correct_answer)
        # make use of maxsplit argument just in case there isanother useful ')'
        incorrect_answers = [option.split(')', 1)[1].lstrip() for option in options if str(option)]
#         print(type(incorrect_answers))
        explanation = exp.replace('Explanation:', '').strip()
    except:
#         print("Invalid format")
        pass
    else:
        quiz = {
            "question": question,
            "incorrect answers": incorrect_answers,
            "correct_answer": correct_answer,
            "explanation": explanation
        }
        questions.append(quiz)
#     print(answer.contents)
#         print(ans, exp)
#     print(type(answer.contents))
#     print(p.prettify())
#     print(p.contents)
#     print(len(p.contents))
pprint.pprint(questions)
print(len(questions))

[{'correct_answer': 'Study of organisms that are not visible to naked eyes',
  'explanation': 'Study of organisms that are not visible to naked eyes is '
                 'known as Microbiology. A special device known as a '
                 'microscope is required to view these organisms. Example: '
                 'Bacteria, viruses, fungi, etc.',
  'incorrect answers': ['Study of molecules that are visible to human eyes',
                        'Study of animals and their family',
                        'Study of microscope'],
  'question': 'What is Microbiology?'},
 {'correct_answer': 'Antoni van Leeuwenhoek',
  'explanation': 'Antoni van Leeuwenhoek, a Delft cloth merchant, is '
                 'considered the father of microbiology. He discovered the '
                 'invisible world of microorganisms using homemade '
                 'microscopes.',
  'incorrect answers': ['Edwin John Butler', 'Ferdinand Cohn', 'Robert Koch'],
  'question': 'Who is known as the father of M

In [7]:
with open('data/microbiology/basic.json', 'w') as file:
    json.dump(questions, file, ensure_ascii=True, indent=4)